<a href="https://colab.research.google.com/github/Jorgecuenca1/Tps_ProcesamientoLenguajeNatural/blob/main/TP%20clase%202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. RULE-BASED NOT CON TF IDF

In [ ]:
# importamos las librerias necesarias
import numpy as np
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings

# ignoramos los warnings
warnings.filterwarnings('ignore')

nltk.download('punkt') # primero descargamos punkt
nltk.download('stopwords') # luego descargamos las stopwords

# esto es solo un ejemplo, deberías cambiar estas respuestas y preguntas por las tuyas
data = {'preguntas': ['¿Cuál es tu nombre?', '¿Cómo estás?', '¿Cuál es tu color favorito?', '¿Dónde vives?'],
        'respuestas': ['Soy un bot', 'Estoy bien, gracias', 'Mi color favorito es azul', 'Vivo en el ciberespacio']}

# convertimos todas las preguntas a minúsculas
data['preguntas'] = [pregunta.lower() for pregunta in data['preguntas']]

# creamos nuestro vectorizador TF-IDF
vectorizer = TfidfVectorizer(ngram_range=(1,3))

# entrenamos el vectorizador con nuestras preguntas
X = vectorizer.fit_transform(data['preguntas'])

def chatbot(pregunta):
    # convertimos la pregunta a minúsculas
    pregunta = pregunta.lower()

    # calculamos la matriz tf-idf de la pregunta
    pregunta_tf_idf = vectorizer.transform([pregunta])

    # calculamos la similitud del coseno entre la pregunta del usuario y todas las preguntas en nuestros datos
    cosineValues = cosine_similarity(pregunta_tf_idf, X)

    # obtenemos el índice de la pregunta más similar en nuestros datos
    idx = np.argmax(cosineValues)

    # si la similitud del coseno es 0 entonces la pregunta del usuario no es similar a ninguna de nuestras preguntas
    if cosineValues[0][idx] == 0:
        return "Lo siento, no puedo responder a esa pregunta.", ""
    else:
        return data['preguntas'][idx], data['respuestas'][idx]

while True:
    userInput = input("Usuario: ")
    if userInput.lower() == 'salir':
        break
    pregunta, respuesta = chatbot(userInput)
    print("Pregunta más cercana: ", pregunta)
    print("ChatBot: ", respuesta)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Usuario: como te llamas?
Pregunta más cercana:  Lo siento, no puedo responder a esa pregunta.
ChatBot:  
Usuario: que eres
Pregunta más cercana:  Lo siento, no puedo responder a esa pregunta.
ChatBot:  
Usuario: hola, como estas?
Pregunta más cercana:  Lo siento, no puedo responder a esa pregunta.
ChatBot:  
Usuario: ¿Cuál es tu nombre?
Pregunta más cercana:  ¿cuál es tu nombre?
ChatBot:  Soy un bot
Usuario: ¿Cómo estás?
Pregunta más cercana:  ¿cómo estás?
ChatBot:  Estoy bien, gracias
Usuario: ¿Cuál es tu color favorito?
Pregunta más cercana:  ¿cuál es tu color favorito?
ChatBot:  Mi color favorito es azul
Usuario: ¿Dónde vives?
Pregunta más cercana:  ¿dónde vives?
ChatBot:  Vivo en el ciberespacio


rule-based bot con DNN

In [ ]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 64
epochs = 100
latent_dim = 256

# Pares de preguntas y respuestas
input_texts = ['Hola', '¿Cómo estás?', 'Adiós', '¿Cuál es tu nombre?', '¿Qué hora es?',
               '¿Puedes ayudarme?', 'No entiendo', '¿Eres un robot?', '¿Dónde estamos?',
               '¿Quién eres?', 'Me siento triste', 'Estoy feliz', 'Estoy aburrido',
               '¿Puedes contarme un chiste?', '¿Cómo es el clima hoy?', '¿Cuál es tu comida favorita?',
               'Estoy aprendiendo Python', 'Me gusta el fútbol', '¿Puedes recomendarme una película?',
               '¿Qué edad tienes?']

target_texts = ['Hola, ¿cómo puedo ayudarte hoy?', 'Soy un bot, no tengo emociones, pero gracias por preguntar. ¿Cómo puedo ayudarte hoy?',
                'Adiós, ¡ten un buen día!', 'Soy Chatbot, un bot de IA.',
                'Lo siento, como bot de chat, no tengo acceso a la hora actual.',
                'Claro, estaré encantado de ayudarte. ¿Qué necesitas?',
                'Lo siento si algo no fue claro. Intenta hacer tu pregunta de otra manera.',
                'Sí, soy un bot de chat basado en inteligencia artificial.',
                'Estamos en el ciberespacio.',
                'Soy Chatbot, un bot de IA desarrollado para responder a tus preguntas.',
                'Lo siento, debes sentirte así. Recuerda que hay personas que pueden ayudarte.',
                '¡Me alegra saber que estás feliz!',
                '¡Puedo ayudarte a aprender algo nuevo si quieres!',
                '¿Por qué los pájaros no usan Facebook? Porque ya tienen Twitter.',
                'Lo siento, como bot de chat, no tengo acceso a información en tiempo real como el clima.',
                'Como bot de IA, no como comida.',
                '¡Eso es genial! Python es un lenguaje de programación muy útil.',
                'Eso es interesante. ¿Te gustaría saber más sobre fútbol?',
                'Lo siento, como un bot de chat, no tengo la capacidad de recomendar películas.',
                'Como un bot de IA, no tengo una edad.']

# Código para preparar los datos de entrenamiento y prueba

input_characters = set()
target_characters = set()

for input_text, target_text in zip(input_texts, target_texts):
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

# Mapa de caracteres a índices/índices a caracteres
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

# Preparación de datos para la LSTM
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

# Creación del modelo
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Entrenamiento del modelo
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=batch_size, epochs=epochs, validation_split=0.2)

# Inferencia
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

# Creación de diccionarios inversos para la inferencia
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char
        if (sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        if sampled_token_index in target_token_index.values():
            target_seq[0, 0, sampled_token_index] = 1.
        states_value = [h, c]
    return decoded_sentence

def get_user_input():
    user_input = input("Por favor, escriba su pregunta: ")
    input_seq = np.zeros((1, max_encoder_seq_length, num_encoder_tokens), dtype='float32')
    for t, char in enumerate(user_input):
        if char in input_token_index:
            input_seq[0, t, input_token_index[char]] = 1.
    return user_input, input_seq

while True:
    user_input, user_input_seq = get_user_input()
    decoded_sentence = decode_sequence(user_input_seq)
    print('Pregunta:', user_input)
    print('Respuesta:', decoded_sentence)


Un bot basado en reglas con TF-IDF, como el primer ejemplo, es relativamente más sencillo y rápido de implementar. Utiliza técnicas clásicas de Procesamiento del Lenguaje Natural (PLN) y Recuperación de Información para responder a las preguntas en función de su similitud con las preguntas predefinidas. Este enfoque tiene la ventaja de ser fácil de entender y de controlar, ya que las respuestas son predefinidas y las reglas están bien establecidas.

En cambio, un bot basado en Redes Neuronales Profundas, como el segundo ejemplo, es más complejo y puede requerir un tiempo de entrenamiento significativo. Sin embargo, tiene la capacidad de aprender y generalizar a partir de los datos de entrenamiento, lo que significa que puede ser capaz de proporcionar respuestas sensatas a preguntas que no se encuentren exactamente en los datos de entrenamiento. Además, este enfoque puede mejorar con el tiempo y con más datos de entrenamiento(esta dificil a mi manera que buen ejemplo el que nos diste en tu clase yo intente algo y esta suepr dificil).

En cuanto a cuál es más óptimo, depende en gran medida del contexto y de los requisitos específicos. Si tienes un conjunto limitado de preguntas y respuestas y prefieres tener un control total sobre las respuestas del bot, un bot basado en reglas con TF-IDF podría ser la opción más adecuada. Por otro lado, si tienes una gran cantidad de datos de entrenamiento y estás dispuesto a invertir tiempo y recursos en el entrenamiento y la afinación de un modelo de aprendizaje profundo, un bot basado en Redes Neuronales Profundas podría proporcionar un rendimiento superior.